# Mysql interaction with python

- 1) Import mysql.connector

In [14]:
import mysql.connector as mysql
import time

- 2) Connects with mysql server

In [15]:
mydb = mysql.connect(
    host="localhost",
    user="root",
    passwd=input("Enter your password: "),
)

- 3) Verify connection and create cursor object. The cursor is basically what interacts with mysql server.

In [16]:
if mydb.is_connected():
    print("Connected to MySQL database")
else:
    print("Connection failed")

cursor = mydb.cursor()

Connected to MySQL database


- 4) It is possible to execute queries using cursor.execute() method. The below example shows how to visualize all the databases in the server.

In [18]:
cursor.execute("SHOW DATABASES")
databases = cursor.fetchall()
for database in databases:
    print(database)

('aggregate',)
('aiaciampino',)
('classicmodels',)
('goshare',)
('impero_romano',)
('information_schema',)
('jobs',)
('mysql',)
('performance_schema',)
('seriea',)
('sys',)
('world',)


- 5) I will now start to work with the database aiaciampino. We first need to drop the database if it already exists. Then we create the database and select it.

In [20]:
cursor.execute("SHOW DATABASES")
databases = cursor.fetchall()
for database in databases:
    if database[0] == "aiaciampino":
        print("Database already exists")
        time.sleep(1)
        cursor.execute("DROP DATABASE aiaciampino")
        mydb.commit()
        print("Database dropped")

print("Creating database...")
time.sleep(1)
cursor.execute("CREATE DATABASE aiaciampino")
print("Database created successfully")





Creating database...
Database created successfully


- 6) We can now create a table. The first table wil be called "teams" and it will have 5 columns: name, city, stadium and number of sanctions. The first column will be the primary key.

-> "CREATE TABLE teams (name VARCHAR(255), city VARCHAR(255), stadium VARCHAR(255), sanctions INT(10), PRIMARY KEY (name));"


In [21]:
cursor.execute("USE aiaciampino")
cursor.execute("SHOW TABLES")
for table in cursor:
    if table[0] == "teams":
        print("Table already exists, dropping it...")
        cursor.execute("DROP TABLE teams")
        time.sleep(1)
        print("Table dropped")
cursor.execute("CREATE TABLE teams (name VARCHAR(255), position INT(10), played INT(10), wins INT(10), draws INT(10), losses INT(10), scored INT(10), conceded INT(10), goal_difference INT(10) , points INT(10), PRIMARY KEY (name));")
cursor.execute("SHOW TABLES")
for table in cursor:
    print(f"Table {table[0]} created") 

Table teams created


- 7) We can now insert data into the table. The following example shows how to insert a single row. Then we will procede by inserting multiple rows at once. First of all, we have to import the dataset and to manipulate the dataframes in order to facilitate the insertion.

In [23]:
import pandas as pd 

data = pd.read_csv(r"Dataset Premier League\points_table.csv")
data_columns = list(data.columns)
data_columns = [data_columns[1]] + [data_columns[0]] + data_columns[2:] # this line is necessary to change the order of the columns
print(data_columns) 
data = data.reindex(columns=data_columns) # this line is necessary to change the order of the columns
data.head()

['Team', 'Pos', 'Pld', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'Pts']


,Team,Pos,Pld,W,D,L,GF,GA,GD,Pts
0,Manchester City,1,38,29,6,3,99,26,73,93
1,Liverpool,2,38,28,8,2,94,26,68,92
2,Chelsea,3,38,21,11,6,76,33,43,74
3,Tottenham Hotspur,4,38,22,5,11,69,40,29,71
4,Arsenal,5,38,22,3,13,61,48,13,69


In [24]:
for element in data.iterrows():
    print("-----------------------")
    print(element[0])
    print("-")
    print(element[1])

-----------------------
0
-
Team    Manchester City
Pos                   1
Pld                  38
W                    29
D                     6
L                     3
GF                   99
GA                   26
GD                   73
Pts                  93
Name: 0, dtype: object
-----------------------
1
-
Team    Liverpool
Pos             2
Pld            38
W              28
D               8
L               2
GF             94
GA             26
GD             68
Pts            92
Name: 1, dtype: object
-----------------------
2
-
Team    Chelsea
Pos           3
Pld          38
W            21
D            11
L             6
GF           76
GA           33
GD           43
Pts          74
Name: 2, dtype: object
-----------------------
3
-
Team    Tottenham Hotspur
Pos                     4
Pld                    38
W                      22
D                       5
L                      11
GF                     69
GA                     40
GD                     29
Pts  

In [25]:
first_team = data.iloc[0].values.tolist()
print(first_team)

try:
    cursor.execute("INSERT INTO teams (name, position, played, wins, draws, losses, scored, conceded, goal_difference, points) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", first_team)
    mydb.commit()
except:
    print("Error while inserting the first team")

cursor.execute("SELECT * FROM teams")
for row in cursor:
    print(row)

try:
    cursor.executemany("INSERT INTO teams (name, position, played, wins, draws, losses, scored, conceded, goal_difference, points) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", data.iloc[1:,:].values.tolist()) 
    mydb.commit()
except:
    print("Error while inserting the other teams")

cursor.execute("SELECT * FROM teams")
for row in cursor:
    print(row)

mydb.commit()


['Manchester City', np.int64(1), np.int64(38), np.int64(29), np.int64(6), np.int64(3), np.int64(99), np.int64(26), np.int64(73), np.int64(93)]
Error while inserting the first team
('Arsenal', 5, 38, 22, 3, 13, 61, 48, 13, 69)
('Aston Villa', 14, 38, 13, 6, 19, 52, 54, -2, 45)
('Brentford', 13, 38, 13, 7, 18, 48, 56, -8, 46)
('Brighton and Hove Albion', 9, 38, 12, 15, 11, 42, 44, -2, 51)
('Burnley', 18, 38, 7, 14, 17, 34, 53, -19, 35)
('Chelsea', 3, 38, 21, 11, 6, 76, 33, 43, 74)
('Crystal Palace', 12, 38, 11, 15, 12, 50, 46, 4, 48)
('Everton', 16, 38, 11, 6, 21, 43, 66, -23, 39)
('Leeds United', 17, 38, 9, 11, 18, 42, 79, -37, 38)
('Leicester City', 8, 38, 14, 10, 14, 62, 59, 3, 52)
('Liverpool', 2, 38, 28, 8, 2, 94, 26, 68, 92)
('Manchester United', 6, 38, 16, 10, 12, 57, 57, 0, 58)
('Newcastle United', 11, 38, 13, 10, 15, 44, 62, -18, 49)
('Norwich City', 20, 38, 5, 7, 26, 23, 84, -61, 22)
('Southampton', 15, 38, 9, 13, 16, 43, 67, -24, 40)
('Tottenham Hotspur', 4, 38, 22, 5, 11, 69,

In [1]:
print("ciao")

ciao
